In [1]:
import pandas as pd
import numpy as np
import datetime
#import orjson
#import json
import pyarrow.feather as feather

# https://dash.plotly.com/installation
# https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
from sqlalchemy import create_engine
import mysql.connector #  pip3 install mysql-connector-python
import configparser as cp

import plotly.express as px
from jupyter_dash import JupyterDash

# import dash_core_components as dcc # deprecated
#import dash_html_components as html
from dash import html,dash_table, dcc
from dash.dependencies import Input,Output
from flask_caching import Cache

In [2]:
host_name='ec2-3-12-83-28.us-east-2.compute.amazonaws.com'

In [5]:
app._terminate_server_for_port(host_name, 8050)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



## Functions 

In [ ]:
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

# Data Loading

## 01 DB connection 

In [ ]:
config=cp.ConfigParser()
config.read('/home/ubuntu/cert/db_login.txt')
db_config=config['ivan_db']


## 2. db connection
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))

## 02 Overall Stock 

### 02.1 last 1 year 

In [ ]:
## Data loading
df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
              con=engine)
df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]

print(df.shape)
print(df.Stock.nunique())
print(df.REFRESH_DATE.max())
df.tail(5)

### 02.2 Last 40 days 

In [ ]:
df_l40=df.loc[df.Date>=df.Date.max()-datetime.timedelta(days=60),:]
print(df_l40.shape)
print(df_l40.Stock.nunique())
print(df_l40.Date.min(),
      df_l40.Date.max()
     )

## 03 Linear regression summary table 

In [ ]:
df_lr=pd.read_sql("""SELECT *
                     FROM STOCK_PRED.LINEAR_REG_L40
                     WHERE MODEL_DATE IN (SELECT MAX(MODEL_DATE) FROM STOCK_PRED.LINEAR_REG_L40)
                     ORDER BY MODEL_DATE DESC, WT_COEF DESC""",
                  con=engine
                 )

## data cleaning
df_lr1=df_lr.loc[:, ['Model_date','Stock','Name','Industry','R_squared','Coef','P_values',
                     'start_price','end_price','Num_records_dist','growth_rate','WT_Coef']]
df_lr1.loc[:,'Name']=[x[:30] for x in df_lr1.Name]
df_lr1.loc[df_lr1.Industry.isnull(),'Industry']='Not Available'
df_lr1.loc[:,'Industry']= [x[:30] for x in df_lr1.Industry]


df_lr1.loc[:,'R_squared']=round(df_lr1.R_squared,2)
df_lr1.loc[:,'Coef']=round(df_lr1.Coef,2)
df_lr1.loc[:,'start_price']=round(df_lr1.start_price,2)
df_lr1.loc[:,'end_price']=round(df_lr1.end_price,2)
df_lr1.loc[:,'growth_rate']=round(df_lr1.growth_rate,2)
df_lr1.loc[:,'P_values']=round(df_lr1.P_values,2)

df_lr1.loc[:,'Model_date']=df_lr1.Model_date.dt.date

df_lr1.drop('Num_records_dist',axis=1,inplace=True)


print(df_lr.shape)
print(df_lr.Stock.nunique())
print(df_lr.groupby('Model_date').size())

print(df_lr1.shape)

In [ ]:
df_lr1.isnull().sum()

In [ ]:
#df_lr.head(5)
df_lr1.Industry

In [ ]:
df_lr_filtered=df_lr.loc[(df_lr.R_squared>0.85)&(df_lr.Coef>0),
                         ['Model_date','Stock','Name','Industry','R_squared','Coef','P_values',
                          'start_price','end_price','Num_records_dist','growth_rate'
                         ]].reset_index(drop=True)
print(df_lr_filtered.shape)
print(df_lr_filtered.Stock.nunique())
df_lr_filtered.head(5)

In [ ]:
## cleaning
df_lr_filtered_1=df_lr_filtered.copy()
df_lr_filtered_1.loc[:,'Name']=[x[:30] for x in df_lr_filtered_1.Name]
df_lr_filtered_1.loc[:,'Industry']=[x[:30] for x in df_lr_filtered_1.Industry]

df_lr_filtered_1.loc[:,'R_squared']=round(df_lr_filtered_1.R_squared,2)
df_lr_filtered_1.loc[:,'Coef']=round(df_lr_filtered_1.Coef,2)
df_lr_filtered_1.loc[:,'start_price']=round(df_lr_filtered_1.start_price,2)
df_lr_filtered_1.loc[:,'end_price']=round(df_lr_filtered_1.end_price,2)
df_lr_filtered_1.loc[:,'growth_rate']=round(df_lr_filtered_1.growth_rate,2)
df_lr_filtered_1.loc[:,'P_values']=round(df_lr_filtered_1.P_values,2)

df_lr_filtered_1.loc[:,'Model_date']=df_lr_filtered_1.Model_date.dt.date

df_lr_filtered_1.drop('Num_records_dist',axis=1,inplace=True)

In [ ]:
df_lr_filtered_1

In [ ]:
px.scatter(df_lr,
           x='Coef',y='R_squared',color='growth_rate',
           title='Sotck Linear Regression Analysis'
                     )

In [ ]:
px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
           x='R_squared',y='Coef',color='growth_rate',size='end_price',
           hover_data=['Stock']  
          )

# Dashboard
- https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e

## import packages 

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
# import dash_core_components as dcc # deprecated
#import dash_html_components as html
from dash import html,dash_table, dcc
from dash.dependencies import Input,Output

In [ ]:
host_name='ec2-3-12-111-80.us-east-2.compute.amazonaws.com'

## Dashboard body - 1

In [ ]:
# buid app
app=JupyterDash(__name__)

fig=px.scatter(df_lr,
           x='R_squared',y='Coef',color='growth_rate',
           title='Sotck Linear Regression Analysis'
                     )

app.layout=html.Div([html.H1('Stock Linear Regression Analysis'),
                     dcc.Graph(id='graph',
                               figure=fig
                              )])





## run app
app.run_server(mode='external',host='ec2-52-14-195-139.us-east-2.compute.amazonaws.com',debug=True)

## Dashboard body -2 

In [ ]:
app=JupyterDash(__name__)

ln_reg_plot=px.scatter(df_lr,
                       x='R_squared',y='Coef'#,size='growth_rate',color='Industry'#
                       ,hover_name='Name'
                      )

app.layout=html.Div([
    html.Div([
        html.H1('Summary Table'),
        dash_table.DataTable(df_lr.head(10).to_dict('records'),
                             [{'name':i, 'id':i} for i in df_lr.columns]
                            )
    ],style={'display':'inline-block'}),
    html.Div([
        html.H1('Coef VS R_squared Distribution'),
        dcc.Graph(figure=ln_reg_plot,style={'display':'inline-block'})
        
    ]),
    html.Div([
        html.Label('Pick Stock'),
        dcc.Dropdown(['CRM','MSFT','APPL'],'CRM')
    ])
    
])

#html.Div(children=[
#        dcc.Graph(id="graph1", style={'display': 'inline-block'}),
#        dcc.Graph(id="graph2", style={'display': 'inline-block'})
#    ])

if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True)
    
    

## Dashboard body -3 

In [ ]:
app=JupyterDash(__name__)


app.layout=html.Div([
    ## left ##
    html.Div([
        dcc.Dropdown(
            options=df_l40.Stock.unique(),
            value='CRM',
            id='pick-stock'
        ),
        html.Div([
            html.H1('Best Performing Stocks'),
            dash_table.DataTable(df_lr_filtered.to_dict('records'),
                                 [{'name':i,'id':i} for i in df_lr_filtered.columns]
                                )
        ],
        style={'displace':'inline-block'}
        ),
        html.Div([
            html.H1('Coef vs R_squared'),
            dcc.Graph(id='coef_r_scatter' ,
                      figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                                        x='R_squared',y='Coef',
                                        color='growth_rate',size='end_price',
                                        hover_data=['Stock']  
                                       )
                      )
        ],
        style={'displace':'inline-block'}
        )
        
    ]),
    
    ## right ##
    html.Div([
        # line chart: last 40 days
        html.Div([
            html.H1('Stock Price Last 40 Days'),
            dcc.Graph(id='stock_price_line_l40')
        ]),
        # line chart: last year
        html.Div([
            html.H1('Stock Price Last Year'),
            dcc.Graph(id='stock_price_line')
        ])
        
    ])
])


@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la

if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )


In [ ]:
if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True)

## Dashboard body - 3.1 

In [ ]:
app=JupyterDash(__name__)


app.layout=html.Div([
    ## left ##
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=df_l40.Stock.unique(),
                value='CRM',
                id='pick-stock')
            
        ]),
        
        html.Div([
            html.Div([
                html.H1('Linear Regression Output'),
                dash_table.DataTable(df_lr_filtered_1.to_dict('records'),
                                 [{'name':i,'id':i} for i in df_lr_filtered_1.columns]
                                )
                
            ]),
            
            html.Div([
                dcc.Graph(id='coef_r_scatter' ,
                          figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                                            x='R_squared',y='Coef',
                                            color='growth_rate',size='end_price',
                                            hover_data=['Stock'],
                                            title='R_squared and Coef Distribution'
                                           )
                      )
                
            ],style={'padding-top':'5px'})
            
            
            
            
            
            
        ],style={'displace':'inline-block','float':'left','padding':'10px 5px'})
        
    ]),
    
    ## right ##
    html.Div([
        dcc.Graph(id='stock_price_line_l40'),
        dcc.Graph(id='stock_price_line') 
        
    ],style={'display':'inline-block','float':'right','width':'49%'})
    
])


@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la

if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )

In [ ]:
len('Universal Logistics Holdings Inc')

## Dashboard - 3.2 

In [ ]:
# 1. adjust format
# 2. add linear regression summary table filters

In [ ]:
app=JupyterDash(__name__)


app.layout=html.Div([
    html.H1('Stock Price Analysis'),
    html.Div('Data as of {:s}'.format(df_lr1.Model_date[0].strftime('%Y-%m-%d'))),
    ## left ##
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=df_l40.Stock.unique(),
                value='CRM',
                id='pick-stock')
        ],style={'display':'inline-block','width':'49%'}),
        
        html.Div([
            dcc.Input(id='r_squared_input',name='R Squared', value=0.8, type='number', placeholder='R_squared'), # R_squared
            dcc.Input(id='coef_input', value=0, type='number', placeholder='Coef'),
            dcc.Input(id='growth_rate_input',value=0, type='number', placeholder='Growth_rate'),
            html.Div(id='filters_input')
        ],style={'display':'inline-block','width':'49%','float':'left'})
    ]),
        

    html.Div([
        html.Div([
            html.H1('Linear Regression Output'),
            dash_table.DataTable(id='linear_reg_summary',
                                 columns=[{'name':i,'id':i} for i in df_lr1.drop('Model_date',axis=1).columns],
                                 page_size=10
                                )
        ]),
        html.Div([
            dcc.Graph(id='coef_r_scatter',
                      figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                                        x='R_squared',y='Coef',
                                        color='growth_rate',size='end_price',
                                        hover_data=['Stock'],
                                        title='R_squared and Coef Distribution'
                                       )
                     )
        ],style={'padding-top':'5px'})
    ],style={'float':'left','padding':'10px 5px','width':'49%'}),
        

    
    ## right ##
    html.Div([
        dcc.Graph(id='stock_price_line_l40'),
        dcc.Graph(id='stock_price_line') 
        
    ],style={'float':'right','width':'49%'})
    
])


@app.callback(
    [Output('linear_reg_summary','data')],
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_linear_reg_tabel(r_squared_thres,coef_thres,growth_rate_thres):
    df_lr1_filtered=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres) ,:].reset_index(drop=True)
    df_lr1_filtered.loc[:,'WT_Coef']=round(df_lr1_filtered.WT_Coef,2)
    df_lr1_filtered.drop('Model_date',axis=1,inplace=True)
    
    return [df_lr1_filtered.to_dict('records')]

@app.callback(
    Output('filters_input','children'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_filters_input(r_squared_thres,coef_thres,growth_rate_thres):
    return 'Filter the data by: R Squared >= {:.2f} & Coefficient >= {:.2f} & Growth Rate >= {:.1f}%'.format(r_squared_thres,
                                                                                                             coef_thres,
                                                                                                             growth_rate_thres*100
                                                                                                            )



@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la

if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )

In [ ]:
help(dcc.Input)

## Dashboard - 3.3 

In [ ]:
# 1.Highlight selected stock in red in scatter plot based on company filter;  > Completed
  ## set those in linear_reg summary table as blue > completed
    
# 2.Configure scatter plot with hover - apply to two line charts > Not possible
  ## conflicts with company filter



In [ ]:
app=JupyterDash(__name__)


app.layout=html.Div([
    html.H1('Stock Price Analysis'),
    html.Div('Data as of {:s}'.format(df_lr1.Model_date[0].strftime('%Y-%m-%d'))),
    ## left ##
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=df_l40.Stock.unique(),
                value='CRM',
                id='pick-stock')
        ],style={'display':'inline-block','width':'49%'}),
        
        html.Div([
            dcc.Input(id='r_squared_input',name='R Squared', value=0.8, type='number', placeholder='R_squared'), # R_squared
            dcc.Input(id='coef_input', value=0, type='number', placeholder='Coef'),
            dcc.Input(id='growth_rate_input',value=0, type='number', placeholder='Growth_rate'),
            html.Div(id='filters_input')
        ],style={'display':'inline-block','width':'49%','float':'left'})
    ]),
        

    html.Div([
        html.Div([
            html.H1('Linear Regression Output - Last 60 Days'),
            dash_table.DataTable(id='linear_reg_summary',
                                 columns=[{'name':i,'id':i} for i in df_lr1.drop('Model_date',axis=1).columns],
                                 page_size=10
                                )
        ]),
        html.Div([
            dcc.Graph(id='coef_r_scatter',
                      #figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                      #                  x='R_squared',y='Coef',
                      #                  color='growth_rate',size='end_price',
                      #                  hover_data=['Stock'],
                      #                  title='R_squared and Coef Distribution'
                      #                 )
                     )
        ],style={'padding-top':'5px'})
    ],style={'float':'left','padding':'10px 5px','width':'49%'}),
        

    
    ## right ##
    html.Div([
        dcc.Graph(id='stock_price_line_l40'),
        dcc.Graph(id='stock_price_line') 
        
    ],style={'float':'right','width':'49%'})
    
])


@app.callback(
    [Output('linear_reg_summary','data')],
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_linear_reg_tabel(r_squared_thres,coef_thres,growth_rate_thres):
    df_lr1_filtered=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres) ,:].reset_index(drop=True)
    df_lr1_filtered.loc[:,'WT_Coef']=round(df_lr1_filtered.WT_Coef,2)
    df_lr1_filtered.drop('Model_date',axis=1,inplace=True)
    
    return [df_lr1_filtered.to_dict('records')]

@app.callback(
    Output('filters_input','children'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_filters_input(r_squared_thres,coef_thres,growth_rate_thres):
    return 'Filter the data by: R Squared >= {:.2f} & Coefficient >= {:.2f} & Growth Rate >= {:.1f}%'.format(r_squared_thres,
                                                                                                             coef_thres,
                                                                                                             growth_rate_thres*100
                                                                                                            )



@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la



@app.callback(
    Output('coef_r_scatter','figure'),
    Input('pick-stock','value'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
    
)
def highlight_scatter_plot(pick_stock,
                           r_squared_thres,coef_thres,growth_rate_thres
                          ):
    
    selected_stocks=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres),'Stock']
    
    df_scatter_clean=df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:].reset_index(drop=True)
    
    fig_scatter=px.scatter(df_scatter_clean,
                           x='R_squared',y='Coef',
                           #color='growth_rate',
                           #size='end_price',
                           #opacity=0.5,
                           hover_data=['Stock'],
                           title='R_squared and Coef Distribution')
    
    #fig_scatter['data'][0]['marker']['color']=['red' if c==pick_stock else 'grey' for c in df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),'Stock']]
    fig_scatter['data'][0]['marker']['color']=np.where([c==pick_stock for c in df_scatter_clean.Stock],'red',
                                                       np.where([c in selected_stocks.values for c in df_scatter_clean.Stock],
                                                                'blue',
                                                                'grey')
                                                      )
    fig_scatter['data'][0]['marker']['size']=[20 if c==pick_stock else 10 for c in df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),'Stock']]
    ## reference: https://community.plotly.com/t/how-to-highlight-a-single-bar-on-select-in-plotly-dash/60739
    ## figure data structure: https://plotly.com/python/figure-structure/
    
    return fig_scatter
    


if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )

## Dashboard - 3.4  Auto-refresh with Cache
- https://community.plotly.com/t/show-and-tell-server-side-caching/42854

In [3]:
from flask_caching import Cache

In [4]:
app=JupyterDash(__name__)

## 01 db connection ##
config=cp.ConfigParser()
config.read('/home/ubuntu/cert/db_login.txt')
db_config=config['ivan_db']


# 2. db connection
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))

## cache - config - start  ##
cache=Cache(app.server,config={
    'CACHE_TYPE':'filesystem',
    'CACHE_DIR':'/home/ubuntu/projects/Stock_Price_Prediction/dashboard/cache_dir/'
})
TIMEOUT = 60*30 # 30 minutes

# cache -1: overall stock price data: there are two df & df_l40 in use, need to consolidate to one
@cache.memoize(timeout=TIMEOUT)
def pull_stock_details():
    df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
                   con=engine)
    df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]
    df.loc[:,'cache_time']=datetime.datetime.now()
    return df.to_json(date_format='iso',orient='split') # split

def stock_details_df():
    return pd.read_json(pull_stock_details(),orient='split')


# cache -2: linear regression results
@cache.memoize(timeout=TIMEOUT)
def pull_reg_result():
    df_lr=pd.read_sql("""SELECT *
                         FROM STOCK_PRED.LINEAR_REG_L40
                         WHERE MODEL_DATE IN (SELECT MAX(MODEL_DATE) FROM STOCK_PRED.LINEAR_REG_L40)
                         ORDER BY MODEL_DATE DESC, WT_COEF DESC""",
                         con=engine
                     )

    ## data cleaning
    df_lr1=df_lr.loc[:, ['Model_date','Stock','Name','Industry','R_squared','Coef','P_values',
                         'start_price','end_price','Num_records_dist','growth_rate','WT_Coef']]
    df_lr1.loc[:,'Name']=[x[:30] for x in df_lr1.Name]
    df_lr1.loc[df_lr1.Industry.isnull(),'Industry']='Not Available'
    df_lr1.loc[:,'Industry']= [x[:30] for x in df_lr1.Industry]


    df_lr1.loc[:,'R_squared']=round(df_lr1.R_squared,2)
    df_lr1.loc[:,'Coef']=round(df_lr1.Coef,2)
    df_lr1.loc[:,'start_price']=round(df_lr1.start_price,2)
    df_lr1.loc[:,'end_price']=round(df_lr1.end_price,2)
    df_lr1.loc[:,'growth_rate']=round(df_lr1.growth_rate,2)
    df_lr1.loc[:,'P_values']=round(df_lr1.P_values,2)

    df_lr1.loc[:,'Model_date']=df_lr1.Model_date.dt.date

    df_lr1.drop('Num_records_dist',axis=1,inplace=True)
    
    return df_lr1.to_json(date_format='iso',orient='split')

def reg_result():
    return pd.read_json(pull_reg_result(),orient='split')

    
    

## cache - config - end ##

app.layout=html.Div([
    html.H1('Stock Price Analysis'),
    html.H3('Stock data as of {}; data cached at {} UTC'.format(stock_details_df().Date.max().date(),
                                                                stock_details_df().cache_time.max().strftime('%Y-%m-%d %H:%M:%S')
                                                               )),
    #html.Div('Data as of {:s}'.format(reg_result()['Model_date'][0])),
    ## left ##
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=reg_result().Stock.unique(),
                value='CRM',
                id='pick-stock')
        ],style={'display':'inline-block','width':'49%'}),
        
        html.Div([
            dcc.Input(id='r_squared_input',name='R Squared', value=0.8, type='number', placeholder='R_squared'), # R_squared
            dcc.Input(id='coef_input', value=0, type='number', placeholder='Coef'),
            dcc.Input(id='growth_rate_input',value=0, type='number', placeholder='Growth_rate'),
            html.Div(id='filters_input')
        ],style={'display':'inline-block','width':'49%','float':'left'})
    ]),
        

    html.Div([
        html.Div([
            html.H1('Linear Regression Output - Last 60 Days'),
            dash_table.DataTable(id='linear_reg_summary',
                                 columns=[{'name':i,'id':i} for i in reg_result().drop('Model_date',axis=1).columns],
                                 page_size=10
                                )
        ]),
        html.Div([
            dcc.Graph(id='coef_r_scatter',
                      #figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                      #                  x='R_squared',y='Coef',
                      #                  color='growth_rate',size='end_price',
                      #                  hover_data=['Stock'],
                      #                  title='R_squared and Coef Distribution'
                      #                 )
                     )
        ],style={'padding-top':'5px'})
    ],style={'float':'left','padding':'10px 5px','width':'49%'}),
        

    
    ## right ##
    html.Div([
        dcc.Graph(id='stock_price_line_l40'),
        dcc.Graph(id='stock_price_line') 
        
    ],style={'float':'right','width':'49%'})
    
])


@app.callback(
    [Output('linear_reg_summary','data')],
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_linear_reg_tabel(r_squared_thres,coef_thres,growth_rate_thres):
    df_lr1=reg_result()
    df_lr1_filtered=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres) ,:].reset_index(drop=True)
    df_lr1_filtered.loc[:,'WT_Coef']=round(df_lr1_filtered.WT_Coef,2)
    df_lr1_filtered.drop('Model_date',axis=1,inplace=True)
    
    return [df_lr1_filtered.to_dict('records')]

@app.callback(
    Output('filters_input','children'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_filters_input(r_squared_thres,coef_thres,growth_rate_thres):
    return 'Filter the data by: R Squared >= {:.2f} & Coefficient >= {:.2f} & Growth Rate >= {:.1f}%'.format(r_squared_thres,
                                                                                                             coef_thres,
                                                                                                             growth_rate_thres*100
                                                                                                            )



@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    df=stock_details_df()
    df_l40=df.loc[df.Date>=df.Date.max()-datetime.timedelta(days=40),:]
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    df=stock_details_df()
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la



@app.callback(
    Output('coef_r_scatter','figure'),
    Input('pick-stock','value'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
    
)
def highlight_scatter_plot(pick_stock,
                           r_squared_thres,coef_thres,growth_rate_thres
                          ):
    df_lr1=reg_result()
    
    selected_stocks=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres),'Stock']
    
    df_scatter_clean=df_lr1.loc[(df_lr1.Coef>-5)&(df_lr1.growth_rate<2),:].reset_index(drop=True)
    
    fig_scatter=px.scatter(df_scatter_clean,
                           x='R_squared',y='Coef',
                           #color='growth_rate',
                           #size='end_price',
                           #opacity=0.5,
                           hover_data=['Stock'],
                           title='R_squared and Coef Distribution')
    
    #fig_scatter['data'][0]['marker']['color']=['red' if c==pick_stock else 'grey' for c in df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),'Stock']]
    fig_scatter['data'][0]['marker']['color']=np.where([c==pick_stock for c in df_scatter_clean.Stock],'red',
                                                       np.where([c in selected_stocks.values for c in df_scatter_clean.Stock],
                                                                'blue',
                                                                'grey')
                                                      )
    fig_scatter['data'][0]['marker']['size']=[20 if c==pick_stock else 10 for c in df_scatter_clean.Stock]
    ## reference: https://community.plotly.com/t/how-to-highlight-a-single-bar-on-select-in-plotly-dash/60739
    ## figure data structure: https://plotly.com/python/figure-structure/
    
    return fig_scatter
    


if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )

Dash app running on http://ec2-3-12-83-28.us-east-2.compute.amazonaws.com:8050/


In [ ]:
def stock_details_df():
    return pd.read_json(pull_stock_details(),orient='split')

In [ ]:
%%time
stock_details_df()

## 3.4.2  Auto-refresh - saved with Feather from Apache Arrow

In [3]:
import pyarrow.feather as feather
from pyarrow import json

In [ ]:
app=JupyterDash(__name__)

## 01 db connection ##
config=cp.ConfigParser()
config.read('/home/ubuntu/cert/db_login.txt')
db_config=config['ivan_db']


# 2. db connection
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))

saved_path='/home/ubuntu/projects/Stock_Price_Prediction/dashboard/cache_dir/feather'

## cache - config - start  ##
cache=Cache(app.server,config={
    'CACHE_TYPE':'FileSystemCache',
    'CACHE_DIR':saved_path
})
TIMEOUT = 60*30 # 30 minutes

# cache -1: overall stock price data: there are two df & df_l40 in use, need to consolidate to one
@cache.memoize(timeout=TIMEOUT)
def pull_stock_details():
    df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
                   con=engine)
    df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]
    df.loc[:,'cache_time']=datetime.datetime.now()
    #return feather.write_feather(df,saved_path+'/df_stock_his_360.feather')
    return df.to_json(date_format='iso',orient='split')

def stock_details_df():
    #return feather.read_feather(saved_path+'/df_stock_his_360.feather')
    return json.read_json(pull_stock_details())


# cache -2: linear regression results
@cache.memoize(timeout=TIMEOUT)
def pull_reg_result():
    df_lr=pd.read_sql("""SELECT *
                         FROM STOCK_PRED.LINEAR_REG_L40
                         WHERE MODEL_DATE IN (SELECT MAX(MODEL_DATE) FROM STOCK_PRED.LINEAR_REG_L40)
                         ORDER BY MODEL_DATE DESC, WT_COEF DESC""",
                         con=engine
                     )

    ## data cleaning
    df_lr1=df_lr.loc[:, ['Model_date','Stock','Name','Industry','R_squared','Coef','P_values',
                         'start_price','end_price','Num_records_dist','growth_rate','WT_Coef']]
    df_lr1.loc[:,'Name']=[x[:30] for x in df_lr1.Name]
    df_lr1.loc[df_lr1.Industry.isnull(),'Industry']='Not Available'
    df_lr1.loc[:,'Industry']= [x[:30] for x in df_lr1.Industry]


    df_lr1.loc[:,'R_squared']=round(df_lr1.R_squared,2)
    df_lr1.loc[:,'Coef']=round(df_lr1.Coef,2)
    df_lr1.loc[:,'start_price']=round(df_lr1.start_price,2)
    df_lr1.loc[:,'end_price']=round(df_lr1.end_price,2)
    df_lr1.loc[:,'growth_rate']=round(df_lr1.growth_rate,2)
    df_lr1.loc[:,'P_values']=round(df_lr1.P_values,2)

    df_lr1.loc[:,'Model_date']=df_lr1.Model_date.dt.date

    df_lr1.drop('Num_records_dist',axis=1,inplace=True)
    
    #return feather.write_feather(df_lr1,saved_path+'/df_stock_lr.feather')
    return df_lr1.to_json(date_format='iso',orient='split')

def reg_result():
    #return feather.read_feather(saved_path+'/df_stock_lr.feather')
    return json.read_json(pull_reg_result())

    
    

## cache - config - end ##

app.layout=html.Div([
    html.H1('Stock Price Analysis'),
    html.H3('Stock data as of {}; data cached at {} UTC'.format(stock_details_df().Date.max().date(),
                                                                stock_details_df().cache_time.max().strftime('%Y-%m-%d %H:%M:%S')
                                                               )),
    #html.Div('Data as of {:s}'.format(reg_result()['Model_date'][0])),
    ## left ##
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=reg_result().Stock.unique(),
                value='CRM',
                id='pick-stock')
        ],style={'display':'inline-block','width':'49%'}),
        
        html.Div([
            dcc.Input(id='r_squared_input',name='R Squared', value=0.8, type='number', placeholder='R_squared'), # R_squared
            dcc.Input(id='coef_input', value=0, type='number', placeholder='Coef'),
            dcc.Input(id='growth_rate_input',value=0, type='number', placeholder='Growth_rate'),
            html.Div(id='filters_input')
        ],style={'display':'inline-block','width':'49%','float':'left'})
    ]),
        

    html.Div([
        html.Div([
            html.H1('Linear Regression Output - Last 60 Days'),
            dash_table.DataTable(id='linear_reg_summary',
                                 columns=[{'name':i,'id':i} for i in reg_result().drop('Model_date',axis=1).columns],
                                 page_size=10
                                )
        ]),
        html.Div([
            dcc.Graph(id='coef_r_scatter',
                      #figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                      #                  x='R_squared',y='Coef',
                      #                  color='growth_rate',size='end_price',
                      #                  hover_data=['Stock'],
                      #                  title='R_squared and Coef Distribution'
                      #                 )
                     )
        ],style={'padding-top':'5px'})
    ],style={'float':'left','padding':'10px 5px','width':'49%'}),
        

    
    ## right ##
    html.Div([
        dcc.Graph(id='stock_price_line_l40'),
        dcc.Graph(id='stock_price_line') 
        
    ],style={'float':'right','width':'49%'})
    
])


@app.callback(
    [Output('linear_reg_summary','data')],
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_linear_reg_tabel(r_squared_thres,coef_thres,growth_rate_thres):
    df_lr1=reg_result()
    df_lr1_filtered=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres) ,:].reset_index(drop=True)
    df_lr1_filtered.loc[:,'WT_Coef']=round(df_lr1_filtered.WT_Coef,2)
    df_lr1_filtered.drop('Model_date',axis=1,inplace=True)
    
    return [df_lr1_filtered.to_dict('records')]

@app.callback(
    Output('filters_input','children'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_filters_input(r_squared_thres,coef_thres,growth_rate_thres):
    return 'Filter the data by: R Squared >= {:.2f} & Coefficient >= {:.2f} & Growth Rate >= {:.1f}%'.format(r_squared_thres,
                                                                                                             coef_thres,
                                                                                                             growth_rate_thres*100
                                                                                                            )



@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    df=stock_details_df()
    df_l40=df.loc[df.Date>=df.Date.max()-datetime.timedelta(days=40),:]
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    df=stock_details_df()
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la



@app.callback(
    Output('coef_r_scatter','figure'),
    Input('pick-stock','value'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
    
)
def highlight_scatter_plot(pick_stock,
                           r_squared_thres,coef_thres,growth_rate_thres
                          ):
    df_lr1=reg_result()
    
    selected_stocks=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres),'Stock']
    
    df_scatter_clean=df_lr1.loc[(df_lr1.Coef>-5)&(df_lr1.growth_rate<2),:].reset_index(drop=True)
    
    fig_scatter=px.scatter(df_scatter_clean,
                           x='R_squared',y='Coef',
                           #color='growth_rate',
                           #size='end_price',
                           #opacity=0.5,
                           hover_data=['Stock'],
                           title='R_squared and Coef Distribution')
    
    #fig_scatter['data'][0]['marker']['color']=['red' if c==pick_stock else 'grey' for c in df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),'Stock']]
    fig_scatter['data'][0]['marker']['color']=np.where([c==pick_stock for c in df_scatter_clean.Stock],'red',
                                                       np.where([c in selected_stocks.values for c in df_scatter_clean.Stock],
                                                                'blue',
                                                                'grey')
                                                      )
    fig_scatter['data'][0]['marker']['size']=[20 if c==pick_stock else 10 for c in df_scatter_clean.Stock]
    ## reference: https://community.plotly.com/t/how-to-highlight-a-single-bar-on-select-in-plotly-dash/60739
    ## figure data structure: https://plotly.com/python/figure-structure/
    
    return fig_scatter
    


if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )

In [ ]:
def pull_stock_details():
    df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
                   con=engine)
    df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]
    df.loc[:,'cache_time']=datetime.datetime.now()
    return feather.write_feather(df,saved_path+'/df_stock_his_360.feather')
def stock_details_df():
    return feather.read_feather(saved_path+'/df_stock_his_360.feather')

In [ ]:
pull_stock_details()

In [ ]:
def pull_stock_details():
    df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 60 DAY""",
                   con=engine)
    df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]
    df.loc[:,'cache_time']=datetime.datetime.now()
    #return feather.write_feather(df,saved_path+'/df_stock_his_360.feather')
    return df.to_json(date_format='iso',orient='split')

def stock_details_df():
    #return feather.read_feather(saved_path+'/df_stock_his_360.feather')
    return json.read_json(pull_stock_details())

In [ ]:
pull_stock_details()

In [ ]:
df_test=stock_details_df()

## 3.4.3 Auto refresh - long callback
* https://community.plotly.com/t/show-and-tell-server-side-caching/42854
* https://stackoverflow.com/questions/69375582/server-side-caching-with-plotly-dash

In [ ]:
## Steps:
# 1. configure CallbackCache > done
# 2. add dcc.Store and connect callbacks
# 3. update callback with @cc.cached_callback() and @cc.callback() > done
# 4. add refresh bottom


In [6]:
from dash_extensions.callback import CallbackCache, DiskCache

In [ ]:
app=JupyterDash(__name__)

## 01 db connection ##
config=cp.ConfigParser()
config.read('/home/ubuntu/cert/db_login.txt')
db_config=config['ivan_db']


# 2. db connection
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))
    
    

## cache - config - end ##

app.layout=html.Div([
    html.H1('Stock Price Analysis'),
    html.H3('Stock data as of {}; data cached at {} UTC'.format(stock_details_df().Date.max().date(),
                                                                stock_details_df().cache_time.max().strftime('%Y-%m-%d %H:%M:%S')
                                                               )),
    
    #html.Div('Data as of {:s}'.format(reg_result()['Model_date'][0])),
    ## left ##
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=reg_result().Stock.unique(),
                value='CRM',
                id='pick-stock')
        ],style={'display':'inline-block','width':'49%'}),
        
        html.Div([
            dcc.Input(id='r_squared_input',name='R Squared', value=0.8, type='number', placeholder='R_squared'), # R_squared
            dcc.Input(id='coef_input', value=0, type='number', placeholder='Coef'),
            dcc.Input(id='growth_rate_input',value=0, type='number', placeholder='Growth_rate'),
            html.Div(id='filters_input')
        ],style={'display':'inline-block','width':'49%','float':'left'})
    ]),
        

    html.Div([
        html.Div([
            html.H1('Linear Regression Output - Last 60 Days'),
            dash_table.DataTable(id='linear_reg_summary',
                                 columns=[{'name':i,'id':i} for i in reg_result().drop('Model_date',axis=1).columns],
                                 page_size=10
                                )
        ]),
        html.Div([
            dcc.Graph(id='coef_r_scatter',
                      #figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                      #                  x='R_squared',y='Coef',
                      #                  color='growth_rate',size='end_price',
                      #                  hover_data=['Stock'],
                      #                  title='R_squared and Coef Distribution'
                      #                 )
                     )
        ],style={'padding-top':'5px'})
    ],style={'float':'left','padding':'10px 5px','width':'49%'}),
        

    
    ## right ##
    html.Div([
        dcc.Loading(dcc.Graph(id='stock_price_line_l40')), # revising dcc.Loading
        dcc.Graph(id='stock_price_line') 
        
    ],style={'float':'right','width':'49%'}),
    
    
    ## loading cache
    dcc.Loading(dcc.Store(id='stock_details_df'),
                fullscreen=True,
                type='dot'
               ),
    dcc.Loading(dcc.Store(id='reg_result'),
                fullscreen=True,
                type='dot'
               )
    
])





## cache - config - start  ##
#cache=Cache(app.server,config={
#    'CACHE_TYPE':'filesystem',
#    'CACHE_DIR':'/home/ubuntu/projects/Stock_Price_Prediction/dashboard/cache_dir/'
#})
#TIMEOUT = 60*30 # 30 minutes

# cache -1: overall stock price data: there are two df & df_l40 in use, need to consolidate to one
#@cache.memoize(timeout=TIMEOUT)

cc=CallbackCache(cache=DiskCache(cache_dir='/home/ubuntu/projects/Stock_Price_Prediction/dashboard/cache_dir/')
                )


@cc.cached_callback(Output('stock_details_df','data'))
def pull_stock_details():
    df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
                   con=engine)
    df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]
    df.loc[:,'cache_time']=datetime.datetime.now()
    return df



# cache -2: linear regression results
@cc.cached_callback(Output('reg_result','data'))
def pull_reg_result():
    df_lr=pd.read_sql("""SELECT *
                         FROM STOCK_PRED.LINEAR_REG_L40
                         WHERE MODEL_DATE IN (SELECT MAX(MODEL_DATE) FROM STOCK_PRED.LINEAR_REG_L40)
                         ORDER BY MODEL_DATE DESC, WT_COEF DESC""",
                         con=engine
                     )

    ## data cleaning
    df_lr1=df_lr.loc[:, ['Model_date','Stock','Name','Industry','R_squared','Coef','P_values',
                         'start_price','end_price','Num_records_dist','growth_rate','WT_Coef']]
    df_lr1.loc[:,'Name']=[x[:30] for x in df_lr1.Name]
    df_lr1.loc[df_lr1.Industry.isnull(),'Industry']='Not Available'
    df_lr1.loc[:,'Industry']= [x[:30] for x in df_lr1.Industry]


    df_lr1.loc[:,'R_squared']=round(df_lr1.R_squared,2)
    df_lr1.loc[:,'Coef']=round(df_lr1.Coef,2)
    df_lr1.loc[:,'start_price']=round(df_lr1.start_price,2)
    df_lr1.loc[:,'end_price']=round(df_lr1.end_price,2)
    df_lr1.loc[:,'growth_rate']=round(df_lr1.growth_rate,2)
    df_lr1.loc[:,'P_values']=round(df_lr1.P_values,2)

    df_lr1.loc[:,'Model_date']=df_lr1.Model_date.dt.date

    df_lr1.drop('Num_records_dist',axis=1,inplace=True)
    
    return df_lr1




## interaction ##

@cc.callback(
    [Output('linear_reg_summary','data')],
    Input('reg_result','data'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_linear_reg_tabel(df_lr1,
                            r_squared_thres,coef_thres,growth_rate_thres):
    df_lr1_filtered=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres) ,:].reset_index(drop=True)
    df_lr1_filtered.loc[:,'WT_Coef']=round(df_lr1_filtered.WT_Coef,2)
    df_lr1_filtered.drop('Model_date',axis=1,inplace=True)
    
    return [df_lr1_filtered.to_dict('records')]

@cc.callback(
    Output('filters_input','children'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_filters_input(r_squared_thres,coef_thres,growth_rate_thres):
    return 'Filter the data by: R Squared >= {:.2f} & Coefficient >= {:.2f} & Growth Rate >= {:.1f}%'.format(r_squared_thres,
                                                                                                             coef_thres,
                                                                                                             growth_rate_thres*100
                                                                                                            )



@cc.callback(
    Output('stock_price_line_l40','figure'),
    Input('stock_details_df','data'), # extract data from cache
    Input('pick-stock','value')
)
def update_l40_trend(df,pick_stock):
    df_l40=df.loc[df.Date>=df.Date.max()-datetime.timedelta(days=40),:]
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@cc.callback(
    Output('stock_price_line','figure'),
    Input('stock_details_df','data'), # extract data from cache
    Input('pick-stock','value')
)
def update_la_trend(df,pick_stock):
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la



@cc.callback(
    Output('coef_r_scatter','figure'),
    Input('reg_result','data'),
    Input('pick-stock','value'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
    
)
def highlight_scatter_plot(df_lr1,
                           pick_stock,r_squared_thres,coef_thres,growth_rate_thres
                          ):
    
    selected_stocks=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres),'Stock']
    
    df_scatter_clean=df_lr1.loc[(df_lr1.Coef>-5)&(df_lr1.growth_rate<2),:].reset_index(drop=True)
    
    fig_scatter=px.scatter(df_scatter_clean,
                           x='R_squared',y='Coef',
                           #color='growth_rate',
                           #size='end_price',
                           #opacity=0.5,
                           hover_data=['Stock'],
                           title='R_squared and Coef Distribution')
    
    #fig_scatter['data'][0]['marker']['color']=['red' if c==pick_stock else 'grey' for c in df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),'Stock']]
    fig_scatter['data'][0]['marker']['color']=np.where([c==pick_stock for c in df_scatter_clean.Stock],'red',
                                                       np.where([c in selected_stocks.values for c in df_scatter_clean.Stock],
                                                                'blue',
                                                                'grey')
                                                      )
    fig_scatter['data'][0]['marker']['size']=[20 if c==pick_stock else 10 for c in df_scatter_clean.Stock]
    ## reference: https://community.plotly.com/t/how-to-highlight-a-single-bar-on-select-in-plotly-dash/60739
    ## figure data structure: https://plotly.com/python/figure-structure/
    
    return fig_scatter


# Registers the callbacks on the application
cc.register(app)

    


if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )

In [7]:
help(dcc.Loading)

Help on class Loading in module dash.dcc.Loading:

class Loading(dash.development.base_component.Component)
 |  Loading(children=None, id=undefined, type=undefined, fullscreen=undefined, debug=undefined, className=undefined, parent_className=undefined, style=undefined, parent_style=undefined, color=undefined, loading_state=undefined, **kwargs)
 |  
 |  A Loading component.
 |  A Loading component that wraps any other component and displays a spinner until the wrapped component has rendered.
 |  
 |  Keyword arguments:
 |  
 |  - children (list of a list of or a singular dash component, string or numbers | a list of or a singular dash component, string or number; optional):
 |      Array that holds components to render.
 |  
 |  - id (string; optional):
 |      The ID of this component, used to identify dash components in
 |      callbacks. The ID needs to be unique across all of the components
 |      in an app.
 |  
 |  - className (string; optional):
 |      Additional CSS class for 

## 3.4.1 Cache with orjson - didn't work

In [ ]:
app=JupyterDash(__name__)

## 01 db connection ##
config=cp.ConfigParser()
config.read('/home/ubuntu/cert/db_login.txt')
db_config=config['ivan_db']


# 2. db connection
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))

## cache - config - start  ##
cache=Cache(app.server,config={
    'CACHE_TYPE':'filesystem',
    'CACHE_DIR':'/home/ubuntu/projects/Stock_Price_Prediction/dashboard/cache_dir/'
})
TIMEOUT = 60*30 # 30 minutes

# cache -1: overall stock price data: there are two df & df_l40 in use, need to consolidate to one
@cache.memoize(timeout=TIMEOUT)
def pull_stock_details():
    df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
                   con=engine)
    df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]
    df.loc[:,'cache_time']=datetime.datetime.now()
    return df.to_json(date_format='iso',orient='split') # split
    #return orjson.dumps(df)

def stock_details_df():
    #return pd.read_json(pull_stock_details(),orient='split')
    return pd.DataFrame(json.loads(pull_stock_details()))

# cache -2: linear regression results
@cache.memoize(timeout=TIMEOUT)
def pull_reg_result():
    df_lr=pd.read_sql("""SELECT *
                         FROM STOCK_PRED.LINEAR_REG_L40
                         WHERE MODEL_DATE IN (SELECT MAX(MODEL_DATE) FROM STOCK_PRED.LINEAR_REG_L40)
                         ORDER BY MODEL_DATE DESC, WT_COEF DESC""",
                         con=engine
                     )

    ## data cleaning
    df_lr1=df_lr.loc[:, ['Model_date','Stock','Name','Industry','R_squared','Coef','P_values',
                         'start_price','end_price','Num_records_dist','growth_rate','WT_Coef']]
    df_lr1.loc[:,'Name']=[x[:30] for x in df_lr1.Name]
    df_lr1.loc[df_lr1.Industry.isnull(),'Industry']='Not Available'
    df_lr1.loc[:,'Industry']= [x[:30] for x in df_lr1.Industry]


    df_lr1.loc[:,'R_squared']=round(df_lr1.R_squared,2)
    df_lr1.loc[:,'Coef']=round(df_lr1.Coef,2)
    df_lr1.loc[:,'start_price']=round(df_lr1.start_price,2)
    df_lr1.loc[:,'end_price']=round(df_lr1.end_price,2)
    df_lr1.loc[:,'growth_rate']=round(df_lr1.growth_rate,2)
    df_lr1.loc[:,'P_values']=round(df_lr1.P_values,2)

    df_lr1.loc[:,'Model_date']=df_lr1.Model_date.dt.date

    df_lr1.drop('Num_records_dist',axis=1,inplace=True)
    
    return df_lr1.to_json(date_format='iso',orient='split')

def reg_result():
    #return pd.read_json(pull_reg_result(),orient='split')
    
    return pd.DataFrame(json.loads(pull_reg_result()))
    

## cache - config - end ##

app.layout=html.Div([
    html.H1('Stock Price Analysis'),
    #html.Div('Data as of {:s}'.format(reg_result()['Model_date'][0])),
    ## left ##
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=reg_result().Stock.unique(),
                value='CRM',
                id='pick-stock')
        ],style={'display':'inline-block','width':'49%'}),
        
        html.Div([
            dcc.Input(id='r_squared_input',name='R Squared', value=0.8, type='number', placeholder='R_squared'), # R_squared
            dcc.Input(id='coef_input', value=0, type='number', placeholder='Coef'),
            dcc.Input(id='growth_rate_input',value=0, type='number', placeholder='Growth_rate'),
            html.Div(id='filters_input')
        ],style={'display':'inline-block','width':'49%','float':'left'})
    ]),
        

    html.Div([
        html.Div([
            html.H1('Linear Regression Output'),
            dash_table.DataTable(id='linear_reg_summary',
                                 columns=[{'name':i,'id':i} for i in reg_result().drop('Model_date',axis=1).columns],
                                 page_size=10
                                )
        ]),
        html.Div([
            dcc.Graph(id='coef_r_scatter',
                      #figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                      #                  x='R_squared',y='Coef',
                      #                  color='growth_rate',size='end_price',
                      #                  hover_data=['Stock'],
                      #                  title='R_squared and Coef Distribution'
                      #                 )
                     )
        ],style={'padding-top':'5px'})
    ],style={'float':'left','padding':'10px 5px','width':'49%'}),
        

    
    ## right ##
    html.Div([
        dcc.Graph(id='stock_price_line_l40'),
        dcc.Graph(id='stock_price_line') 
        
    ],style={'float':'right','width':'49%'})
    
])


@app.callback(
    [Output('linear_reg_summary','data')],
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_linear_reg_tabel(r_squared_thres,coef_thres,growth_rate_thres):
    df_lr1=reg_result()
    df_lr1_filtered=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres) ,:].reset_index(drop=True)
    df_lr1_filtered.loc[:,'WT_Coef']=round(df_lr1_filtered.WT_Coef,2)
    df_lr1_filtered.drop('Model_date',axis=1,inplace=True)
    
    return [df_lr1_filtered.to_dict('records')]

@app.callback(
    Output('filters_input','children'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
)
def update_filters_input(r_squared_thres,coef_thres,growth_rate_thres):
    return 'Filter the data by: R Squared >= {:.2f} & Coefficient >= {:.2f} & Growth Rate >= {:.1f}%'.format(r_squared_thres,
                                                                                                             coef_thres,
                                                                                                             growth_rate_thres*100
                                                                                                            )



@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    df=stock_details_df()
    df_l40=df.loc[df.Date>=df.Date.max()-timedelta(days=40),:]
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    df=stock_details_df()
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la



@app.callback(
    Output('coef_r_scatter','figure'),
    Input('pick-stock','value'),
    Input('r_squared_input','value'),
    Input('coef_input','value'),
    Input('growth_rate_input','value')
    
)
def highlight_scatter_plot(pick_stock,
                           r_squared_thres,coef_thres,growth_rate_thres
                          ):
    df_lr1=reg_result()
    
    selected_stocks=df_lr1.loc[(df_lr1.R_squared>=r_squared_thres)&
                               (df_lr1.Coef>=coef_thres)&
                               (df_lr1.growth_rate>=growth_rate_thres),'Stock']
    
    df_scatter_clean=df_lr1.loc[(df_lr1.Coef>-5)&(df_lr1.growth_rate<2),:].reset_index(drop=True)
    
    fig_scatter=px.scatter(df_scatter_clean,
                           x='R_squared',y='Coef',
                           #color='growth_rate',
                           #size='end_price',
                           #opacity=0.5,
                           hover_data=['Stock'],
                           title='R_squared and Coef Distribution')
    
    #fig_scatter['data'][0]['marker']['color']=['red' if c==pick_stock else 'grey' for c in df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),'Stock']]
    fig_scatter['data'][0]['marker']['color']=np.where([c==pick_stock for c in df_scatter_clean.Stock],'red',
                                                       np.where([c in selected_stocks.values for c in df_scatter_clean.Stock],
                                                                'blue',
                                                                'grey')
                                                      )
    fig_scatter['data'][0]['marker']['size']=[20 if c==pick_stock else 10 for c in df_scatter_clean.Stock]
    ## reference: https://community.plotly.com/t/how-to-highlight-a-single-bar-on-select-in-plotly-dash/60739
    ## figure data structure: https://plotly.com/python/figure-structure/
    
    return fig_scatter
    


if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )

In [ ]:
app._terminate_server_for_port(host_name, 8050)

In [ ]:
df.to_json(date_format='iso',orient='split')